### Types récursifs

Là où on comprend toute la richesse des types de `OCaml`, c'est quand on utilise la récursivité : en effet, un type peut figurer dans sa définition. L'exemple le plus emblématique est celui des listes. Une liste c'est soit vide, soit un maillon contenant un élément puis le reste de la liste. Ainsi, on pourra définir le type des listes d'entiers ainsi :

In [1]:
type liste_entiers = Vide | Cons of int * liste_entiers

type liste_entiers = Vide | Cons of int * liste_entiers

La liste qui contient 1 puis 2 puis 3 correspond alors à la valeur

```ocaml
Cons(1, Cons(2, Cons(3, Vide)))
```

Pour écrire une fonction on peut faire un filtra comme précédemment :

```ocaml
let est_vide l =
    match l with
    | Vide -> true
    | Cons(x, q) -> false
```

mais comme on va souvent utiliser la valeur correspondant au type, presque touts les fonctions seront récursives. Pour calculer la longueur de la liste, on pourra alors faire :

```ocaml
let rec longueur l =
    match l with
    | Vide -> 0
    | Cons(x, q) -> 1 + longueur q
```

In [2]:
let rec somme l =
    match l with 
    | Vide -> 0
    | Cons(x, q) -> x + somme q
    (* Renvoie la somme des entiers dans l *)

val somme : liste_entiers -> int = <fun>

In [3]:
assert (somme Vide = 0);
assert (somme (Cons(1,Vide)) = 1);
assert (somme (Cons(1, Cons(2, Vide))) = 3)

- : unit = ()

Pour la question suivante, comme pour d'autres questions à venir, il est nécessaire de pouvoir produire une erreur quand on ne peut pas calculer quelque chose. On utilise pour cela un mécanisme en `OCaml` appelé *exceptions*. Pour le moment, on ne rentre pas dans les détails, mais on retiendra juste que pour pouvoir faire une erreur il suffit d'écrire

```ocaml
failwith "message a afficher"
```

Une fonction comme

```ocaml
let f x = failwith "erreur"
```

a le type `'a -> 'b` mais c'est illusion car on ne pourra jamais calculer de valeur avec vu qu'elle produit une erreur de manière inconditionnelle.

In [6]:
let rec nieme l n =
    match l with
    | Vide -> failwith "No element"
    | Cons(x,q) -> if n=0 then x else nieme q (n-1)
    
    (* Renvoie le n element de la liste en partant de 0 pour le premier element.
    Produit une erreur si la liste comporte moins de n+1 element *)

val nieme : liste_entiers -> int -> int = <fun>

In [7]:
let l = Cons(1, Cons(2, Cons(5, Vide))) in
assert (nieme l 0 = 1);
assert (nieme l 1 = 2);
assert (nieme l 2 = 5);
assert(try let _ = nieme l 4 in false with _ -> true)

- : unit = ()

# Les listes en `OCaml`

En fait, le type des listes est tellement important qu'il est défini directment dans la syntaxe de `OCaml`. Ainsi on a un type `'a list` avec pour liste vide `[]` à la place de `Vide` et `t::q` à la place de `Cons(t,q)`.

On pourra donc écrire `1::2::3::[]` mais comme c'est assez courant de définir des valeurs ainsi, on peut utiliser directement le raccourci `[1;2;3]` (**Attention** aux `;` et non des `,` comme en `Python`. Si on se trompe, on va implicitement créer des listes de tuple).

La fonction `longueur` pourra alors s'écrire

```ocaml
let rec longueur l =
    match l with
    | [] -> 0
    | t::q -> 1 + longueur q
```

## Premières fonctions

In [9]:
let tete (l : 'a list) : 'a =
    match l with
    | [] -> failwith "Liste vide"
    | t::q -> t
    (* Renvoie le premier élément de la liste l ou produit une erreur avec
    failwith "Liste vide" si elle vide *)

val tete : 'a list -> 'a = <fun>

In [12]:
let queue (l : 'a list) : 'a list =
    match l with
    | [] -> failwith "Liste vide"
    | t::q -> q
    (* Renvoie la queue de la liste l, c'est à dire la liste q telle que l=t::q
    si jamais l est vide produit la même erreur que tete *)

val queue : 'a list -> 'a list = <fun>

In [14]:
let l = [1;2;3] in assert(tete l :: queue l = l);
assert(try let _ = tete [] in false with _ -> true);
assert(try let _ = queue [] in false with _ -> true)

- : unit = ()

In [16]:
let rec somme (l : int list) : int =
    match l with
    | [] -> 0
    | t::q -> t + somme q 
    (* renvoie la somme des élements de la liste d'entiers l *)

val somme : int list -> int = <fun>

In [18]:
assert(somme [] = 0);
assert(somme [1] = 1);
assert(somme [1;2;3;4] = 10)

- : unit = ()

In [20]:
let rec nieme (l : 'a list) (n : int) : 'a =
    match l with 
    | [] -> failwith "No element!:("
    | t::q -> if n=0 then t else nieme q (n-1)
    (* renvoie le n-ieme élément de l en partant de 0
    produit une erreur si l contient moins de n+1 
    éléments (n'utilisez pas longueur !) *)

val nieme : 'a list -> int -> 'a = <fun>

In [22]:
assert(nieme [1;2;5] 0 = 1);
assert(nieme [1;2;5] 1 = 2);
assert(nieme [1;2;5] 2 = 5);
assert(try let _ = nieme [1;2;5] 3 in false with _ -> true)

- : unit = ()

In [24]:
let rec contient (x:'a) (l:'a list) : bool =
    match l with
    | [] -> false
    | t::q -> if t=x then true else contient x q
    (* renvoie true si et seulement si x est un élément de la liste *)

val contient : 'a -> 'a list -> bool = <fun>

In [26]:
assert(contient 0 [1;2;0;3]);
assert(not (contient 4 [1;2;3;5]));
assert(not (contient 0 []));
assert(contient 0 [0])

- : unit = ()

In [48]:
let rec range (a:int) (b:int) : int list =
    if a<b then a::(range (a+1) b)
    else []
    (* renvoie la liste des entiers a, a+1, ..., b-1 comme
    on peut le faire avec Python *)

val range : int -> int -> int list = <fun>

In [50]:
assert(range 0 4 = [0;1;2;3]);
assert(range 1 0 = []);
assert(range 5 5 = []);
assert(range (-2) 0 = [-2;-1])

- : unit = ()

In [52]:
let rec ajoute_fin (x : 'a) (l : 'a list) : 'a list =
    match l with
    | [] -> [x]
    | t::q -> t::(ajoute_fin x  q)
    (* Renvoie la liste obtenue en rajoutant x à la fin de la liste l *)

val ajoute_fin : 'a -> 'a list -> 'a list = <fun>

In [54]:
assert(ajoute_fin 1 [] = [1]);
assert(ajoute_fin 2 [1;3;5] = [1;3;5;2])

- : unit = ()

In [56]:
let rec concat (l1 : 'a list) (l2 : 'a list) : 'a list =
    match l1 with
    | [] -> l2
    | t::q -> t::(concat q l2)
    (* Renvoie la liste obtenue en collant l2 à la fin de l1. Voir exemples *)

val concat : 'a list -> 'a list -> 'a list = <fun>

In [58]:
assert(concat [] [1;2] = [1;2]);
assert(concat [1;2] [] = [1;2]);
assert(concat [1;2] [3;4] = [1;2;3;4])

- : unit = ()

## Manipulations fonctionnelles des listes

On fait référence ici à des fonctions génériques sur les listes utilisant des fonctions comme argument.

In [60]:
let rec applique (f : 'a -> 'b) (l : 'a list) : 'b list =
    match l with
    | [] -> []
    | t::q -> (f t)::(applique f q)
    (* applique la fonction f sur chaque élément de la liste l.
    si l = [a1; a2; ...; an] alors applique f l renvoie
    [f a1; f a2; ...; f an] *)

val applique : ('a -> 'b) -> 'a list -> 'b list = <fun>

In [62]:
assert(applique (fun n -> n+1) [1;2;3] = [2;3;4]);
assert(applique (fun x -> x) [] = []);
assert(applique int_of_string ["1";"2";"3"] = [1;2;3])

- : unit = ()

In [69]:
let rec existe (f : 'a -> bool) (l : 'a list) : bool =
    match l with
    | [] -> false
    | t::q -> if f t then true else existe f q
    (* renvoie true si et seulement si un élément x de l est tel que f x = true *)

val existe : ('a -> bool) -> 'a list -> bool = <fun>

In [72]:
assert(existe (fun n -> n mod 2 = 0) [1;3;2;5]);
assert(not (existe (fun n -> n mod 2 = 0) [1;3;5]));
assert(not (existe (fun x -> x) []))

- : unit = ()

In [78]:
let rec forall (f : 'a -> bool) (l : 'a list) : bool =
    match l with
    | [] -> true
    | t::q -> if f t then forall f q else false
(* Écrire une fonction forall sur le modele de existe qui renvoie true si et seulement si 
TOUS les éléments de l vérifient f x = true.
On pourra utiliser la fonction précédente ou en écrire une ex nihilo. *)


val forall : ('a -> bool) -> 'a list -> bool = <fun>

In [80]:
assert(forall (fun n -> n mod 2 = 0) [2;4;6;10]);
assert(not (forall (fun n -> n mod 2 = 0) [2;4;6;11;10]));
assert(forall (fun x -> x) [])

- : unit = ()

In [94]:
let rec filtre_options (l : 'a option list) : 'a list =
    match l with
    | [] -> []
    | None::q ->  filtre_options q 
    | (Some t)::q -> t::(filtre_options q)
    (* Prend une liste l de 'a option et renvoie la liste obtenue en supprimant tous les None
    et en renvoyant pour chaque Some x la valeur x. Voir les exemples ci-dessous *)

val filtre_options : 'a option list -> 'a list = <fun>

In [96]:
assert(filtre_options [None; None; None] = []);
assert(filtre_options [Some 3; None; Some 5] = [3; 5])

- : unit = ()

Une opération importante, complémentaire de `applique` qui applique une fonction sur chaque elément, est `fold` qui effectue un calcul à partir de chaque élément.

Si `l = [a1; a2; ...; an]` alors un `fold` à gauche avec la fonction `f` et la valeur initiale `b` va calculer

$$f(...f(f(b,a_1), a_2), ...), a_n)$$

et un `fold` à droite

$$f(a_1, f(a_2, ... f(a_{n-1},f(a_n,b))...)$$


In [132]:
let rec fold_gauche (f : 'a -> 'b -> 'a) (b : 'a) (l : 'b list) : 'a =
    match l with 
    |[] -> b
    |t::q -> (fold_gauche f (f b t) q) 
    (* Calcule le fold à gauche de f sur l avec la valeur initiale b *)

val fold_gauche : ('a -> 'b -> 'a) -> 'a -> 'b list -> 'a = <fun>

In [134]:
fold_gauche (^) "" ["1";"2";"3"]

- : string = "123"

In [135]:
let l = [1;2;3;4] in
assert(fold_gauche (+) 0 l = 10);
assert(fold_gauche (max) 0 l = 4);
assert(fold_gauche (min) 10 l = 1);
assert(fold_gauche (fun s i -> s ^ string_of_int i) "" l = "1234");
assert(fold_gauche (fun a b -> a) 42 [] = 42)

- : unit = ()

## Le module `List`

En fait, la plupart des fonctions qu'on vient de définir existe déjà dans la bibliothèque standard au sein du **module** `List`. Un module est un regroupement de définitions de types et de fonctions qui peut ensuite être appelés directement (sous réserve d'avoir chargé le binaire correspondant, mais c'est le cas pour la bibliothèque standard).

Ainsi il y a dans le module `List` une fonction `mem` qui est équivalente à `contient`. Pour l'appeler sur une liste `l` et un élément `x` on écrit `List.mem x l`.

On peut trouver l'intégralité des fonctions sur la page https://ocaml.org/api/List.html. Voici une correspondance avec les fonctions qu'on a défini au dessus :

* `nieme` est `List.nth`, avec une version renvoyant une option `List.nth_opt`
* `tete` est `List.hd` et `queue` est `List.tl`
* `append` est `List.append`, auquel on peut aussi accéder avec `@`. Ainsi `l1@l2` appelle `List.append l1 l2`
* `existe` est `List.exists` et `pourtout` est `List.forall`
* `applique` est `List.map`
* `fold_gauche` est `List.fold_left`, il existe aussi `List.fold_right`

# FRACTRAN

Un programme FRACTRAN est une liste de fractions $P = (\frac{p_1}{q_1}, \dots, \frac{p_n}{q_n})$ dans $\mathbb{Q}_+^*$.

Une entrée pour un programme est un entier $n \in \mathbb{N}^*$.

On exécute un tel programme ainsi :

* Tant qu'il existe un denominateur $q_i$ tel que $q_i | n$.
    * Soit $\frac{p_{i_0}}{q_{i_0}}$ la fraction de plus petit indice telle que $q_{i_0} | n$.
    * On pose $n = n' q_{i_0}$
    * On remplace $n \leftarrow n' p_{i_0}$
* on renvoie $n$ qui est la sortie du programme.

Si on considère $P = (\frac{455}{33}, \frac{11}{13}, \frac{1}{11},
\frac{3}{7}, \frac{11}{2}, \frac{1}{3})$ et $n = 72$, on va avoir :

* $\frac{11}{2}$ est la première fraction et on pose donc $n = 36 \times 11 = 396$
* $\frac{455}{33}$ est la première fraction et on pose donc $n = 12 \times 455 = 5460$
* ...

On continue alors jusquà sortir de la boucle avec $n = 15625$.

On va représenter les fractions $\frac{p}{q}$ par des couples `(p,q)` d'entiers. Un programme
est alors une liste de couple (un élément du type `(int * int) list`.

In [145]:
type fraction = int * int
type programme = fraction list

let p = [ (17,91); (78,85); (19,51); (23,38);
      (29,33); (77,29); (95,23); (77,19);
      (1,17); (11,13); (13,11); (15,2); (1,7); (55,1) ]

type fraction = int * int
type programme = fraction list
val p : (int * int) list =
  [(17, 91); (78, 85); (19, 51); (23, 38); (29, 33); (77, 29); (95, 23);
   (77, 19); (1, 17); (11, 13); (13, 11); (15, 2); (1, 7); (55, 1)]

In [153]:
let rec premiere_fraction (l : programme) (n : int) : fraction option =
    match l with
    | [] -> None
    | (a,b)::q when n mod b = 0 -> Some (a,b)
    | t::q -> premiere_fraction q n  
    (* Renvoie Some (a,b) quand a/b est la première fraction du programme telle que b divise n.
    Renvoie None s'il n'y en a pas. *)

val premiere_fraction : programme -> int -> fraction option = <fun>

In [154]:
assert(premiere_fraction p 2 = Some (15,2));
assert(premiere_fraction p 235 = Some (55, 1));
assert(premiere_fraction [ (3,4) ] 5 = None)

- : unit = ()

In [162]:
let rec execute (l : programme) (n : int) (etapes_max : int) : int list =
    if etapes_max=0 then [] else
    match (premiere_fraction l n) with
    | None -> []
    | Some (p,q) -> let n' = p*(n/q) in n'::(execute l n' (etapes_max-1))
        
    (* execute le programme l en accumulant tous les entiers produit
    à chaque étape dans une liste qu'on renverra.
    Le nombre maximum d'etapes, c'est-à-dire la taille maximale de la liste est donnée
    par etapes_max. ATTENTION à le prendre en compte pour ne pas faire de boucles 
    infinies si le programme ne termine pas *)

val execute : programme -> int -> int -> int list = <fun>

In [163]:
assert(execute p 2 100 = [15; 825; 725; 1925; 2275; 425; 390; 330; 290; 770; 910; 170; 
 156; 132; 116; 308; 364; 68; 4; 30; 225; 12375; 10875; 28875; 25375; 67375; 79625;
 14875; 13650; 2550; 2340; 1980; 1740; 4620; 4060; 10780; 12740; 2380; 2184; 408;
 152; 92; 380; 230; 950; 575; 2375; 9625; 11375; 2125; 1950; 1650; 1450;
 3850; 4550; 850; 780; 660; 580; 1540; 1820; 340; 312; 264; 232; 616; 728;
 136; 8; 60; 450; 3375; 185625; 163125; 433125; 380625; 1010625; 888125;
 2358125; 2786875; 520625; 477750; 89250; 81900; 15300; 14040; 11880; 10440;
 27720; 24360; 64680; 56840; 150920; 178360; 33320; 30576; 5712; 2128; 1288;
 5320]);
assert(execute [(3,2)] 4 10 = [6; 9])

- : unit = ()

In [164]:
execute [455,33;11,13;1,11;3,7;11,2;1,3] 72 1000

- : int list =
[396; 5460; 4620; 63700; 53900; 4900; 2100; 900; 4950; 68250; 57750; 796250;
 673750; 61250; 26250; 11250; 61875; 853125; 721875; 9953125; 8421875;
 765625; 328125; 140625; 46875; 15625]

Comme les variables locales peuvent contenir des fonctions, cela permet de définir des fonctions auxilliares. Si on considère la fonction `fibo` qui permet de calculer la suite de Fibonnaci efficacement à l'aide d'une autre fonction `fiboC`.

```ocaml
let rec fiboC n =
    if n = 0 then (0, 1)
    else let a, b = fiboC (n-1) in
         (b, a+b)

let fibo n =
    let a, b = fiboC n in
    a
```

Grâce aux variables locales, on peut *cacher* `fiboC` à l'intérieur de `fibo` :

```ocaml
let fibo n =
    let rec fiboC n =
        if n = 0 then (0,1)
        else let a, b = fiboC (n-1) in
            (b, a+b)
    in
    let a, b = fiboC n in
    a
```

Personne ne peut alors accéder à la fonction `fiboC` en dehors de `fibo`. Cela permet de travailler avec des fonctions ayant parfois plus d'arguments tout en offrant une interface d'appel simple.

In [167]:
let res_de_p n =
    let rec puis a b=
        match b with
        |0 -> 1
        |b -> a*puis(b-1);
    let valuation2 n =
        let p=ref 0 in 
        while n land (puis 2 p) && (puis 2 p)>n do
            p := !p +1
        done;
        if n=(puis 2 !p) then Some p
        else None
    List.map (fun n->(valuation2 n)<>None) (execute p 2 n)
    
        
    (* renvoie la liste des exposants des puissances de 2 présentes dans l'exécution 
    de p sur 2 en n étapes maximum.
    On pourra définir une fonction auxilliaire valuation2 : int -> int option
    telle que valuation2 n renvoie None si n n'est pas une puissance de 2 
    et Some p si n = 2^p. On pourra ensuite utiliser une fonction définie 
    précédemment. *)

Line 13, characters 13-14:
Error: Syntax error


In [25]:
assert(res_de_p 15000 = [2; 3; 5; 7; 11]);
assert(res_de_p 100 = [2; 3])